# 导入包

In [1]:
from collections import defaultdict
from tqdm import tqdm
import re
import json
from copy import deepcopy
import random

import torch
import os
import pickle
from transformers import AutoTokenizer
import transformers
import torch
import torch.nn as nn
import math
from transformers import AutoModel
import pandas as pd
import numpy as np
from nlp_basictasks.tasks import Ner
from nlp_basictasks.evaluation import nerEvaluator
from nlp_basictasks.readers.ner import InputExample
from collections import Counter
from copy import deepcopy
from nlp_basictasks.modules.transformers import BertTokenizer
device=torch.device('cuda')

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


# 预测测试数据中的topic实体

## 加载测试数据

In [80]:
def read_data(data_path):
    with open(data_path) as f:
        lines=f.readlines()
    data=[]
    for line in lines:
        data.append(json.loads(line))
    print("原始数据有{}个样本".format(len(data)))
    return data
    
data=[]
data.extend(read_data('/home/xhsun/Desktop/KG/kgCLUE/qa_data/test.json'))

原始数据有3000个样本


In [81]:
data[0]

{'id': 0, 'question': '我很好奇刘质平的老师是谁？'}

## 加载NER模型

In [2]:
ner_model_path='/home/xhsun/NLP/KGQA/KG/kgCLUE/NER-Match/NER/'
ner_model=Ner(ner_model_path,label2id=None,is_finetune=True,use_crf=True)
ner_model._target_device='cuda'

2021-12-21 09:14:54 - INFO - __init__ - 43 : Load label2id from /home/xhsun/NLP/KGQA/KG/kgCLUE/NER-Match/NER/label2id.json
2021-12-21 09:14:54 - INFO - __init__ - 53 : The label2id is
 {"[PAD]": 0, "O": 1, "B-ent": 2, "I-ent": 3}
2021-12-21 09:14:54 - INFO - get_config_dict - 177 : loading configuration file /home/xhsun/NLP/KGQA/KG/kgCLUE/NER-Match/NER/BERT/config.json
2021-12-21 09:14:54 - INFO - from_pretrained - 404 : loading bert model file /home/xhsun/NLP/KGQA/KG/kgCLUE/NER-Match/NER/BERT
2021-12-21 09:14:54 - INFO - from_pretrained - 423 : BertConfig has been loaded from /home/xhsun/NLP/KGQA/KG/kgCLUE/NER-Match/NER/BERT/config.json
2021-12-21 09:14:57 - INFO - from_pretrained - 125 : loading vocabulary file /home/xhsun/NLP/KGQA/KG/kgCLUE/NER-Match/NER/BERT/vocab.txt
2021-12-21 09:14:57 - INFO - __init__ - 274 : Loading model from /home/xhsun/NLP/KGQA/KG/kgCLUE/NER-Match/NER/, which has been finetuned.
2021-12-21 09:14:57 - INFO - __init__ - 70 : Use pytorch device: cuda
2021-12-2

In [11]:
query="我很好奇刘质平的老师是谁？"
sentence_list=ner_model.model.tokenizer.tokenize(query)
ner_model.predict([sentence_list],input_is_list=True,input_is_string=False)

  0%|          | 0/1 [00:00<?, ?it/s]

[['O', 'O', 'O', 'O', 'B-ent', 'I-ent', 'I-ent', 'O', 'O', 'O', 'O', 'O', 'O']]

In [4]:
sentence_list

['<', '人', '类', '群', '星', '闪', '耀', '时', '>', 'isbn', '码']

In [12]:
def predict_topic_entity(ner_model,query):
    sentence_list=ner_model.model.tokenizer.tokenize(query)
    predict_label_list=ner_model.predict([sentence_list],input_is_string=False,input_is_list=True)[0]
    assert len(predict_label_list)==len(sentence_list)
    topic_entity=''
    for i in range(len(sentence_list)):
        token=sentence_list[i]
        predict_label=predict_label_list[i]
        if predict_label!='O':
            topic_entity+=token
    topic_entity=topic_entity.replace('##','')
    return topic_entity

In [13]:
predict_topic_entity(ner_model,query=query)

  0%|          | 0/1 [00:00<?, ?it/s]

'刘质平'

In [9]:
import numpy as np

In [10]:
a=np.random.randn(3,2)
a

array([[ 1.36623312, -0.8321556 ],
       [-1.11883503, -0.09603982],
       [-0.44734909, -0.21182716]])

In [14]:
np.max(a[:,1])

-0.09603981576940618

In [15]:
np.argmax(a[:,1])

1

## 预测topic实体

In [82]:
def predict_topic_entity(ner_model,query):
    predict_label_list=ner_model.predict(query)[0]
    sentence_list=ner_model.model.tokenizer.tokenize(query)
    assert len(predict_label_list)==len(sentence_list)
    topic_entity=''
    for i in range(len(sentence_list)):
        token=sentence_list[i]
        predict_label=predict_label_list[i]
        if predict_label!='O':
            topic_entity+=token
    return topic_entity

all_questions=[example['question'] for example in data]
all_predict_heads=ner_model.predict(all_questions)

  0%|          | 0/94 [00:00<?, ?it/s]

In [83]:
print(all_questions[0],all_predict_heads[0])

我很好奇刘质平的老师是谁？ ['O', 'O', 'O', 'O', 'B-ent', 'I-ent', 'I-ent', 'O', 'O', 'O', 'O', 'O', 'O']


In [84]:
for i in range(len(data)):
    predict_label_list=all_predict_heads[i]
    sentence_list_by_tokenizer=ner_model.model.tokenizer.tokenize(data[i]['question'])
    sentence_list=list(data[i]['question'])
    if len(sentence_list_by_tokenizer)!=len(sentence_list):
        assert len(sentence_list)>len(sentence_list_by_tokenizer)
        sentence_list=sentence_list_by_tokenizer
    assert len(predict_label_list)==len(sentence_list)
    topic_entity=''
    for k in range(len(sentence_list)):
        token=sentence_list[k]
        predict_label=predict_label_list[k]
        if predict_label!='O':
            topic_entity+=token
    topic_entity=topic_entity.replace('##','')
    data[i].update({'topic_entity':topic_entity})

In [85]:
data

[{'id': 0, 'question': '我很好奇刘质平的老师是谁？', 'topic_entity': '刘质平'},
 {'id': 1, 'question': '请问南关町的平假名是什么？', 'topic_entity': '南关町'},
 {'id': 2, 'question': '请问新道寺女子高中的平假名是什么？', 'topic_entity': '新道寺女子高中'},
 {'id': 3, 'question': '海部市的平假名是什么？', 'topic_entity': '海部市'},
 {'id': 4, 'question': '八代市的平假名是什么？', 'topic_entity': '八代市'},
 {'id': 5, 'question': '三泽市的平假名叫什么？', 'topic_entity': '三泽市'},
 {'id': 6, 'question': '芥川龙之介奖的平假名叫什么？', 'topic_entity': '芥川龙之介奖'},
 {'id': 7, 'question': '尤莉的日本字怎么写啊？', 'topic_entity': '尤莉'},
 {'id': 8, 'question': '临海女子高中的日本字怎么写啊？', 'topic_entity': '临海女子高中'},
 {'id': 9, 'question': '我很好奇子实体是什么构造的？', 'topic_entity': '子实体'},
 {'id': 10, 'question': '我很好奇涨紧轮是什么构造的？', 'topic_entity': '涨紧轮'},
 {'id': 11, 'question': '你知道卡板的构造吗？', 'topic_entity': '卡板'},
 {'id': 12, 'question': '你知道苑里车站的构造吗？', 'topic_entity': '苑里车站'},
 {'id': 13, 'question': '你知道急流槽的主要构成吗？', 'topic_entity': '急流槽'},
 {'id': 14, 'question': '你知道煌斑岩的主要构成吗？', 'topic_entity': '煌斑岩'},
 {'id': 15, 'question': '涅墨西斯

## 分析不明确的topic实体

In [86]:
def ParsingEntity(question,topic_entity):
    try:
        assert len(topic_entity)>=5
        start_token=topic_entity[:2].lower()#前两个字符
        end_token=topic_entity[-2:].lower()#后两个字符
        copy_question=question.lower()
        pattern=start_token+'(.*)'+end_token
        result=re.search(pattern=pattern,string=copy_question)
        start_id,end_id=result.span()
        return question[start_id:end_id]
    except:
        return ''
    
count=0
for i in range(len(data)):
    question,topic_entity=data[i]['question'],data[i]['topic_entity']
    if topic_entity not in question:
        if topic_entity.upper() in question:
            data[i]['topic_entity']=topic_entity.upper()
        elif topic_entity[0].upper()+topic_entity[1:] in question:
            data[i]['topic_entity']=topic_entity[0].upper()+topic_entity[1:]
        elif ParsingEntity(question,topic_entity=topic_entity)!='':
            data[i]['topic_entity']=ParsingEntity(question,topic_entity=topic_entity)
        else:
            count+=1
            print(data[i])

{'id': 367, 'question': "A Hard Day's Night的制作是什么？", 'topic_entity': "ahardday'snight"}
{'id': 759, 'question': '书本里的蚂蚁的尺寸是多少？', 'topic_entity': '书里的蚂蚁'}
{'id': 1074, 'question': 'Black Swallowtail是在龙珠的哪个剧集里面出现的？', 'topic_entity': 'blackswallowtail珠'}
{'id': 1231, 'question': '鱻的五笔86&98是什么？', 'topic_entity': '[UNK]'}
{'id': 1797, 'question': '(35396) 1997 Xf11的ISBN是什么？', 'topic_entity': '(35396)1997xf11'}
{'id': 1946, 'question': 'Chinese Physics B的编辑部在哪', 'topic_entity': 'chinesephysicsb'}
{'id': 2122, 'question': '上党梆子非物质遗产级别是什么', 'topic_entity': '上党梆子质'}
{'id': 2241, 'question': '告诉我7 days的原唱好吗', 'topic_entity': '7days'}
{'id': 2242, 'question': '告诉我9 to 5的原唱好吗', 'topic_entity': '9to5'}
{'id': 2377, 'question': '乐山机场航站楼面积又多大', 'topic_entity': '乐山机场站'}
{'id': 2409, 'question': '丁字双髻鲨的同义学名是什么', 'topic_entity': '丁双髻鲨'}
{'id': 2456, 'question': '你知道B A FIGHTER的首演时间吗', 'topic_entity': 'bafighter'}
{'id': 2573, 'question': '玩游戏的大佬们，你知道D.Va的定位是啥吗', 'topic_entity': 'd.va'}
{'id': 2624, 'que

In [87]:
count

16

<font color=red>经过多次筛选，最终3000条仅有16条样本的实体识别结果无法处理</font>

In [89]:
data[367]['topic_entity']="A Hard Day's Night"
data[759]['topic_entity']='书本里的蚂蚁'
data[1074]['topic_entity']='Black Swallowtail'
data[1231]['topic_entity']='鱻'
data[1797]['topic_entity']='(35396) 1997 Xf11'
data[1946]['topic_entity']='Chinese Physics B'
data[2122]['topic_entity']='上党梆子'
data[2241]['topic_entity']='7 days'
data[2242]['topic_entity']='9 to 5'
data[2377]['topic_entity']='乐山机场'
data[2409]['topic_entity']='丁字双髻鲨'
data[2456]['topic_entity']="B A FIGHTER"
data[2573]['topic_entity']='D.Va'
data[2624]['topic_entity']='Gliese 876 d'
data[2670]['topic_entity']='Chuck D'
data[2936]['topic_entity']='AKB48团体组合成员'

In [90]:
for example in data:
    question,topic_entity=example['question'],example['topic_entity']
    assert topic_entity in question

**现在提交的测试集中所有问题的topic_entity已经全部正确找出**

In [91]:
# with open("../qa_data/private_test.json",'w') as f:
#     for example in data:
#         f.write(json.dumps(example,ensure_ascii=False)+'\n')

In [92]:
private_test_data=deepcopy(data)

# 将数据处理成QA对形式

## 读取原始数据

In [93]:
def read_data(data_path):
    with open(data_path) as f:
        lines=f.readlines()
    data=[]
    for line in lines:
        data.append(json.loads(line))
    print("原始数据有{}个样本".format(len(data)))
    return data
    
data=[]
data.extend(read_data('/home/xhsun/Desktop/KG/kgCLUE/qa_data/train.json'))
data.extend(read_data('/home/xhsun/Desktop/KG/kgCLUE/qa_data/dev.json'))
data.extend(read_data('/home/xhsun/Desktop/KG/kgCLUE/qa_data/test_public.json'))

原始数据有18000个样本
原始数据有2000个样本
原始数据有2000个样本


In [94]:
print("原始数据有{}个样本".format(len(data)))
for i in range(10):
    print(data[i])

原始数据有22000个样本
{'id': 0, 'question': '你知道守望星光是哪家公司出品的吗？', 'answer': '守望星光（韩玉玲、刘东考演唱歌曲） ||| 出品公司 ||| 韩玉玲原创音乐'}
{'id': 1, 'question': '武汉交通职业学院计算机协会有哪些学校？', 'answer': '武汉交通职业学院计算机协会 ||| 学校 ||| 武汉交通职业学院'}
{'id': 2, 'question': '你知道龙晶用繁体字怎么写吗？', 'answer': '龙晶（游戏） ||| 繁体 ||| 龍晶'}
{'id': 3, 'question': '无罪无我的和声是什么？', 'answer': '无罪无我 ||| 和声 ||| 章韵、徐晨辰'}
{'id': 4, 'question': '你知道影舞者的车身有多重吗？', 'answer': '影舞者（《QQ飞车》中的赛车） ||| 车身重量 ||| 1232kg'}
{'id': 5, 'question': '冯慧专业是什么方向？', 'answer': '冯慧（武汉大学教授） ||| 专业方向 ||| 偏微分方程数值解'}
{'id': 6, 'question': '你知道戴姆勒股份公司有多少员工吗？', 'answer': '戴姆勒股份公司 ||| 员工数 ||| 298655 人(2020年)'}
{'id': 7, 'question': '谁知道魔力w.i.t.c.h.系列有哪些丛书？', 'answer': '魔力w.i.t.c.h. ||| 丛书系列 ||| 魔力w.i.t.c.h.100少女完全手册、魔力w.i.t.c.h.故事集、魔力w.i.t.c.h.小说《破碎的魔球》'}
{'id': 8, 'question': '谁知道端午节有什么重要意义？', 'answer': '端午节（中国四大传统节日之一） ||| 节日意义 ||| 传承与弘扬非物质文化'}
{'id': 9, 'question': '谁唱的风不息？', 'answer': '风不息（1995年郭富城发行的音乐专辑） ||| 歌手 ||| 郭富城'}


## 将原始数据转换为QA对的形式

In [95]:
all_examples=[]
bad_examples=[]
for example in data:
    question,answer=example['question'],example['answer']
    triplet=answer.strip().split('|||')
    assert len(triplet)==3
    assert type(question)==str
    question=question.strip()
    entity=triplet[0].strip()
    rel=triplet[1].strip()
    ans=triplet[2].strip()
    
    if ('（' in entity and '）' in entity) and entity not in question:
        #print(entity,question)
        #assert entity.split('（')[0] in question
        question=question.replace(entity.split('（')[0],entity)
            
    if entity in question:
        start_id=question.find(entity)
        end_id=start_id+len(entity)
        question=question[:start_id]+'<'+entity+'>'+question[end_id:]
        all_examples.append({"question":question,'topic_entity':entity,'answer':ans,'relation':rel})
    else:
        bad_examples.append(example)

In [96]:
print(len(data),len(all_examples))
print('转换后的每一个样本形如：')
for i in range(5):
    print(all_examples[i])

22000 21975
转换后的每一个样本形如：
{'question': '你知道<守望星光（韩玉玲、刘东考演唱歌曲）>是哪家公司出品的吗？', 'topic_entity': '守望星光（韩玉玲、刘东考演唱歌曲）', 'answer': '韩玉玲原创音乐', 'relation': '出品公司'}
{'question': '<武汉交通职业学院计算机协会>有哪些学校？', 'topic_entity': '武汉交通职业学院计算机协会', 'answer': '武汉交通职业学院', 'relation': '学校'}
{'question': '你知道<龙晶（游戏）>用繁体字怎么写吗？', 'topic_entity': '龙晶（游戏）', 'answer': '龍晶', 'relation': '繁体'}
{'question': '<无罪无我>的和声是什么？', 'topic_entity': '无罪无我', 'answer': '章韵、徐晨辰', 'relation': '和声'}
{'question': '你知道<影舞者（《QQ飞车》中的赛车）>的车身有多重吗？', 'topic_entity': '影舞者（《QQ飞车》中的赛车）', 'answer': '1232kg', 'relation': '车身重量'}


In [97]:
for i in range(len(bad_examples)):
    print(bad_examples[i])

{'id': 209, 'question': '哈维尔埃尔南德斯习惯用哪只脚？', 'answer': '哈维尔·埃尔南德斯（1988年生墨西哥足球运动员） ||| 惯用脚 ||| 右脚'}
{'id': 747, 'question': '正大综艺是什么频道播出的？', 'answer': '2009年正大综艺节目列表 ||| 播出频道 ||| 中央电视台综艺频道'}
{'id': 838, 'question': '有人了解网易的股票代码吗？', 'answer': '网易公司 ||| 股票代码 ||| NTES（美股）'}
{'id': 977, 'question': '安徽日报农村版的周期是什么？', 'answer': '安徽日报·农村版 ||| 周期 ||| 周二刊'}
{'id': 2798, 'question': '你知道利群集团的年营业额是多少吗？', 'answer': '利群集团股份有限公司 ||| 年营业额 ||| 3038679 万元(2019年)'}
{'id': 4147, 'question': '你知道我家的女仆有够烦的出版社是什么吗？', 'answer': '我家的女仆有够烦！（中村カンコ著作的漫画） ||| 其他出版社 ||| 青文出版社（中国台湾）'}
{'id': 4148, 'question': '仓央嘉措是在哪里逝世的？', 'answer': '第六世达赖喇嘛·仓央嘉措 ||| 逝世地 ||| 中国青海湖附近公噶瑙尔'}
{'id': 5046, 'question': '请问阿莱克斯库珀哪只脚为惯用脚？', 'answer': '阿莱克斯·库珀 ||| 惯用脚 ||| 左脚'}
{'id': 5358, 'question': '认真和我谈恋爱的游戏平台是什么呀?', 'answer': '认真和我谈恋爱！！（2009年みなとそふと开发的恋爱冒险电脑游戏） ||| 游戏平台 ||| PC(Windows2000/XP/Vista)'}
{'id': 5591, 'question': '姚明的中文名是什么？', 'answer': '王子（亚洲篮球联合会主席、中国篮球协会主席） ||| 中文名 ||| 王子'}
{'id': 5988, 'question': '电视里演的还珠格格是哪种类型的电视剧？', 

**问题中的实体(topic entity)用<>括起来**

# 根据构造的QA对数据来提取一个小规模的知识图谱
提交的测试集泛化性不高

## 首先读取大规模的知识图谱

In [3]:
sub_map = defaultdict(list)
obj_map = defaultdict(list)
so_map=defaultdict(list)
sp_map=defaultdict(list)
kg_path='../knowledge/Knowledge.txt'
with open(kg_path) as f:
    lines=f.readlines()
    
entities_set=set()
relations_set=set()
print(len(lines))

#new_triples=[]
for i in tqdm(range(len(lines))):
    line=lines[i]
    l = line.strip().split('\t')
    s = l[0].strip()
    p = l[1].strip()
    o = l[2].strip()
    if s=='' or p=='' or o=='':
        print('-'*100)
        continue
    entities_set.add(s)
    entities_set.add(o)
    relations_set.add(p)
    #new_triples.append((s,p,o))
    sub_map[s].append((p, o))
    obj_map[o].append((s, p))
    so_map[(s,o)].append(p)
    sp_map[(s,p)].append(o)

22883549


100%|███████████████████████████████████████████████████████████████████████████████████████| 22883549/22883549 [01:30<00:00, 254008.77it/s]


In [9]:
# kg_path='../knowledge/Knowledge.txt'
# with open(kg_path,'w') as f:
#     for triples in new_triples:
#         f.write('\t'.join(triples)+'\n')

In [5]:
print(len(sp_map))

19743901


In [8]:
len(sub_map)

3121457

In [9]:
with open("../knowledge/head_entities.dict",'w') as f:
    for ent in sub_map.keys():
        f.write(str(ent)+"\n")

In [99]:
print('大规模图谱中一共有{}个三元组，{}个实体，{}个关系'.format(len(lines),len(entities_set),len(relations_set)))

大规模图谱中一共有22883549个三元组，9418104个实体，245837个关系


**sub_map记录的是每一个头实体(subject、head)对应的所有可能出现的(关系,尾实体)的集合**

In [100]:
sub_map['周杰伦']

[('中文名', '周杰伦'),
 ('主要成就',
  '获得十五座金曲奖两届台湾金曲奖最佳国语男歌手四届世界音乐大奖最畅销中华区艺人入选美国CNN亚洲25位最具影响力人物\xa0《Fast Company》全球百大创意人物大中华区8次年度唱片销量冠军2010年MP3下载量全球第三2009年悉尼演唱会票房公告牌全球第二两届MTV亚洲大奖最受欢迎艺人连续7年获得IFPI香港唱片销量大奖十大销量国语唱片四届新加坡金曲奖最受欢迎男歌手收起'),
 ('主要成就',
  '获得十五座金曲奖两届台湾金曲奖最佳国语男歌手四届世界音乐大奖最畅销中华区艺人入选美国CNN亚洲25位最具影响力人物\xa0《Fast Company》全球百大创意人物展开获得十五座金曲奖两届台湾金曲奖最佳国语男歌手四届世界音乐大奖最畅销中华区艺人入选美国CNN亚洲25位最具影响力人物\xa0《Fast Company》全球百大创意人物大中华区8次年度唱片销量冠军2010年MP3下载量全球第三2009年悉尼演唱会票房公告牌全球第二两届MTV亚洲大奖最受欢迎艺人连续7年获得IFPI香港唱片销量大奖十大销量国语唱片四届新加坡金曲奖最受欢迎男歌手收起'),
 ('代表作品', '星晴、龙卷风、简单爱、双截棍、晴天、以父之名、七里香、夜曲、青花瓷、彩虹、稻香、告白气球'),
 ('出生地', '台湾省新北市'),
 ('出生日期', '1979年1月18日'),
 ('别名', '周董'),
 ('国籍', '中国'),
 ('外文名', 'Jay Chou'),
 ('影视代表作', '不能说的秘密、逆战、天台'),
 ('影视代表作', '青蜂侠、头文字D、大灌篮、寻找周杰伦'),
 ('星座', '摩羯座'),
 ('毕业院校', '淡江中学'),
 ('民族', '汉族'),
 ('经纪公司', '杰威尔音乐有限公司'),
 ('职业', '歌手、音乐人、制作人、导演、商人'),
 ('血型', 'O型'),
 ('身高', '175cm')]

**obj_map记录的是每一个尾实体(object、tail)对应的所有可能出现的(头实体，关系)的集合**

In [101]:
obj_map['金庸']

[('七虫七花膏', '发明者'),
 ('三恋', '编剧'),
 ('三恋', '编剧'),
 ('九阴真经（金庸小说中的武学秘籍）', '作者'),
 ('书剑恩仇录（1976年香港TVB郑少秋主演版电视剧）', '原著'),
 ('书剑恩仇录（1976年香港TVB郑少秋主演版电视剧）', '原著'),
 ('书剑恩仇录（1981年香港邵氏狄龙主演版电影）', '原著'),
 ('书剑恩仇录（1981年香港邵氏狄龙主演版电影）', '原著'),
 ('书剑恩仇录（2015张纪中版书剑恩仇录）', '原著'),
 ('书剑恩仇录（2015张纪中版书剑恩仇录）', '原著'),
 ('书剑恩仇录（原著改编连环画）', '小说作者'),
 ('书剑恩仇录（金庸创作长篇武侠小说）', '作者'),
 ('书剑恩仇录（金庸创作长篇武侠小说）', '作者'),
 ('伏魔杖法', '作者'),
 ('何三七', '作者'),
 ('何三七', '作者'),
 ('侠客行（金庸创作长篇武侠小说）', '作者'),
 ('侠客行（金庸创作长篇武侠小说）', '作者'),
 ('俞连舟', '作者'),
 ('俞连舟', '作者'),
 ('倚天屠龙记之魔教教主', '编剧'),
 ('倚天屠龙记（1986年王天林监制的TVB电视剧）', '原著'),
 ('倚天屠龙记（1994年台湾台视版马景涛主演电视剧）', '原著'),
 ('倚天屠龙记（1994年马景涛主演电视剧）', '原著'),
 ('倚天屠龙记（2001年香港TVB版吴启华主演电视剧）', '原著'),
 ('倚天屠龙记（2001年香港TVB版吴启华主演电视剧）', '原著'),
 ('倚天屠龙记（2003年合拍版苏有朋主演电视剧）', '原著'),
 ('倚天屠龙记（2003年合拍版苏有朋主演电视剧）', '原著'),
 ('倚天屠龙记（2009年大陆版邓超主演电视剧）', '原著'),
 ('倚天屠龙记（2009年大陆版邓超主演电视剧）', '原著'),
 ('倚天屠龙记（2019年曾舜晞主演电视剧）', '原著'),
 ('倚天屠龙记（2019年蒋家骏执导电视剧）', '原著'),
 ('倚天屠龙记（共四册）', '作者'),
 ('倚天屠龙记（武侠小说改编漫画）', '原作'),
 ('倚天屠龙记（金庸创

**so_map记录的每一个（头实体，尾实体）对应的所有可能的关系集合**

In [102]:
print(so_map[('周杰伦', '汉族')])
print(so_map[('金庸', '中国')])

['民族']
['国籍']


# 确保问答对中的topic实体与answer在KG中

In [109]:
print(len(all_examples))
for i,example in enumerate(all_examples):
    question,topic_entity,answer=example['question'],example['topic_entity'],example['answer']
    assert topic_entity in entities_set
    try:
        assert answer in entities_set
    except:
        print(example)
        del all_examples[i]
print(len(all_examples))

21974
21974


## 确保预测的提交测试集的每一个topic_entity在KG中

In [107]:
def ScreenEntity(topic_entity):
    if topic_entity[1:] in entities_set:
        return topic_entity[1:]
    elif topic_entity[2:] in entities_set:
        return topic_entity[2:]
    elif topic_entity[:-1] in entities_set:
        return topic_entity[:-1]
    elif topic_entity[:-2] in entities_set:
        return topic_entity[:-2]
    else:
        return ''
    
for i,example in enumerate(private_test_data):
    question,topic_entity=example['question'],example['topic_entity']
    assert topic_entity in question
    if topic_entity not in entities_set:
        topic_entity=ScreenEntity(topic_entity=topic_entity)
        if topic_entity!='':
            assert topic_entity in question and topic_entity in entities_set
            private_test_data[i]['topic_entity']=topic_entity
            
for i,example in enumerate(private_test_data):
    question,topic_entity=example['question'],example['topic_entity']
    assert topic_entity in question
    if topic_entity not in entities_set:
        print(example)

{'id': 374, 'question': '谁知道进取性优生学叫什么？', 'topic_entity': '进取性优生'}
{'id': 838, 'question': '1945-1949年苏-35战斗机生产了多少台？', 'topic_entity': '1945-1949年苏-35战斗机'}
{'id': 839, 'question': '1945-1949年玉神工艺品有限公司生产了多少台？', 'topic_entity': '1945-1949年玉神工艺品有限公司'}
{'id': 930, 'question': '你知道奈鲁布恩坎的壳需要多少等级吗？', 'topic_entity': '奈鲁布恩坎'}
{'id': 1394, 'question': '企业国有产权无偿划转工作指引出自哪里？', 'topic_entity': '企业国有产权无偿划转工作'}
{'id': 1803, 'question': '0-3岁好习惯儿歌有什么ISBN有谁知道吗', 'topic_entity': '0-3岁好习惯'}
{'id': 1835, 'question': '一壹国际影视文化传媒在那个地方地方注册登记的', 'topic_entity': '一壹国际影视文化传媒'}
{'id': 1992, 'question': '41824183次列车的长度是多少', 'topic_entity': '41824183次'}
{'id': 2033, 'question': "(It's not a crime) It's just what we do的导演是谁", 'topic_entity': "It's not a crime) It's just what we do"}
{'id': 2034, 'question': '(You Drive Me) Crazy的导演是谁', 'topic_entity': 'You Drive Me) Crazy'}
{'id': 2036, 'question': '...Ready for It?的MV导演是那个', 'topic_entity': '...Ready for It?的MV导演是那个'}
{'id': 2064, 'question': '那些集里面有一字剑', 'topic_en

In [108]:
with open("../qa_data/private_test.json",'w') as f:
    for example in private_test_data:
        f.write(json.dumps(example,ensure_ascii=False)+'\n')

经过多次screen，3000条测试数据中已保证2983条数据中的topic实体是可以在KG中找到的

已经确保QA数据集中的topic实体和答案可以在知识图谱中找得到
# 保存实体和关系

In [13]:
with open('../knowledge/entities.dict','w') as f:
    for i,entity in enumerate(entities_set):
        if entity.strip()=='':
            continue
        f.write(str(entity)+'\t'+str(i)+'\n')
with open('../knowledge/relations.dict','w') as f:
    for i,rel in enumerate(relations_set):
        if rel.strip()=='':
            continue
        f.write(str(rel)+'\t'+str(i)+'\n')

In [14]:
len(entities_set)

9418104

In [15]:
len(relations_set)

245837

# 保存QA数据集

In [16]:
print(len(all_examples))
def write_file(data,path):
    f=open(path,'w')
    for example in data:
        question,topic_entity,answer,relation=example['question'],example['topic_entity'],example['answer'],example['relation']
        question=question.replace('\t','')
        answer=answer.replace('\t','')
        assert topic_entity in entities_set
        assert answer in entities_set
        f.write(question+'\t'+answer+'\t'+'\n')
    f.close()
    
write_file(all_examples[:-2000],'/home/xhsun/Desktop/KG/kgCLUE/qa_data/train.txt')
write_file(all_examples[-2000:],'/home/xhsun/Desktop/KG/kgCLUE/qa_data/test.txt')

21974


## 根据QA对数据构造小规模知识图谱

符号说明
- KG   知识图谱               
- $T_{big}$  大规模KG中三元组集合 
- $E_{big}$  大规模KG中的实体集合   
- $R_{big}$  大规模KG中的关系集合   

构造的思路是对于数据集中每一个（问题，答案）：
1. 将问题中的topic entity作为头实体，答案视为尾实体，根据（头实体，尾实体）的组合从so_map中找到所有可能的关系。
2. 根据问题中的topic entity，获取这个topic entity的n跳路径内的所有实体（n一般设置为2-4即可）

第一步的目的是统计QA数据集中每一个（问题，答案）之间的所有关系，记为集合$R_{qa}$，该集合代表的是整个QA数据集中所有可能出现的关系。
针对$T_{big}$，我们仅保留涉及这些关系的三元组，**也就是说对于$T_{big}$中的某个三元组$(h,r,t)$，如果$r$不在集合$R_{qa}$中,那么这个三元组就会被舍弃**。

显然这一步的目的是为了缩小关系集合，从而进一步的缩小三元组数量以及对应的实体数量。

第二步的目的是通过限定路径长度进一步的缩小实体数量。

In [13]:
small_triplets=set()
small_entities=set()
small_relations=set()
new_examples=[]
bad_entities=[]
bad_answers=[]
for i in tqdm(range(len(all_examples))):
    example=deepcopy(all_examples[i])
    question,topic_entity,answer,relation=example['question'],example['topic_entity'],example['answer'],example['relation']
    answer=answer.replace('\t','').replace(' ','')
    example['answer']=answer
    if answer.strip()=='':
        continue
    elif topic_entity not in sub_map:
        bad_entities.append(topic_entity)
        continue
    elif answer not in obj_map:
        bad_answers.append(answer)
        continue
    else:
        head=topic_entity
        small_entities.add(head)
        small_entities.add(answer)
        for rel in so_map[(head,answer)]:
            small_relations.add(rel)#仅仅添加qa数据对中涉及的关系即可
        
        for p1, o1 in sub_map[head]:
            if p1 in small_relations:
                small_triplets.add((head,p1,o1))
                small_entities.add(o1)
            for p2, o2 in sub_map[o1]:
                if p2 in small_relations:
                    small_triplets.add((o1,p2,o2))
                    small_entities.add(o2)
                for p3, o3 in sub_map[o2]:
                    if p3 in small_relations:
                        small_triplets.add((o2,p3,o3))
                        small_entities.add(o3)
                    for p4, o4 in sub_map[o3]:
                        if p4 in small_relations:
                            small_triplets.add((o3,p4,o4))
                            small_entities.add(o4)
        new_examples.append(example)

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 21975/21975 [00:02<00:00, 9093.93it/s]


In [14]:
print('大规模图谱中一共有{}个三元组，{}个实体，{}个关系'.format(len(lines),len(entities_set),len(relations_set)))
print('小规模图谱中一共有{}个三元组，{}个实体，{}个关系'.format(len(small_triplets),len(small_entities),len(small_relations)))

大规模图谱中一共有22884223个三元组，9418365个实体，245838个关系
小规模图谱中一共有324494个三元组，210227个实体，2274个关系


**QA数据集和知识图谱构造完毕**

In [15]:
new_examples[0]

{'question': '你知道<守望星光（韩玉玲、刘东考演唱歌曲）>是哪家公司出品的吗？',
 'topic_entity': '守望星光（韩玉玲、刘东考演唱歌曲）',
 'answer': '韩玉玲原创音乐',
 'relation': '出品公司'}

In [24]:
for example in new_examples:
    topic_entity,answer,relation=example['topic_entity'],example['answer'],example['relation']
    assert topic_entity.strip() in small_entities
    assert answer.strip() in small_entities
    #assert relation in small_relations

# 将QA数据集划分并保存

In [16]:
print(len(new_examples))
def write_file(data,path):
    f=open(path,'w')
    for example in data:
        question,topic_entity,answer,relation=example['question'],example['topic_entity'],example['answer'],example['relation']
        if answer=='' or answer.strip()=='':
            count+=1
            continue
        f.write(question+'\t'+answer+'\t'+'\n')
    f.close()
    
write_file(new_examples[:-2000],'../../train.txt')
write_file(new_examples[-2000:],'../../test.txt')

20724


# 保存构造的小规模知识图谱

In [17]:
print('小规模图谱中一共有{}个三元组，{}个实体，{}个关系'.format(len(small_triplets),len(small_entities),len(small_relations)))

小规模图谱中一共有324494个三元组，210227个实体，2274个关系


## 保存三元组

In [18]:
with open("../../KgCLUE-main/knowledge/small_kg.txt",'w') as f:
    for head,rel,tail in small_triplets:
        f.write('|||'.join([head,rel,tail])+'\n')

## 保存实体

In [19]:
with open("../../KgCLUE-main/knowledge/small_knowledge/entities.dict",'w') as f:
    for i,ent in enumerate(small_entities):
        f.write(ent+'\t'+str(i)+'\n')

## 保存关系

In [20]:
with open("../../KgCLUE-main/knowledge/small_knowledge/relations.dict",'w') as f:
    for i,rel in enumerate(small_relations):
        f.write(rel+'\t'+str(i)+'\n')

**至此，数据处理部分完毕**

In [9]:
with open("../../KgCLUE-main/knowledge/entities.dict",'w') as f:
    for i,ent in enumerate(entities_set):
        f.write(ent+'\t'+str(i)+'\n')

In [13]:
with open("../../KgCLUE-main/knowledge/relations.dict",'w') as f:
    for i,ent in enumerate(relations_set):
        if ent=='' or ent=='\n':
            continue
        f.write(ent+'\t'+str(i)+'\n')

In [21]:
bad_count=0
with open("../../KgCLUE-main/knowledge/Knowledge.txt",'w') as f:
    for i in tqdm(range(len(lines))):
        triplets='|||'.join(lines[i].strip().split('\t'))
        if len(triplets.split('|||'))!=3:
            bad_count+=1
            continue
        f.write(triplets+'\n')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22884223/22884223 [00:18<00:00, 1233481.41it/s]


In [22]:
bad_count

0

In [23]:
len(lines)

22884223

In [32]:
import os

ent2id = {}
with open(os.path.join('../../KgCLUE-main/knowledge/', 'entities.dict')) as f:
    lines=f.readlines()
for i in tqdm(range(len(lines))):
    l = lines[i].strip().split('\t')
    ent2id[l[0].strip()] = len(ent2id)

rel2id = {}
with open(os.path.join('../../KgCLUE-main/knowledge/', 'relations.dict')) as f:
    lines=f.readlines()
for i in tqdm(range(len(lines))):
    l = lines[i].strip().split('\t')
    rel2id[l[0].strip()] = int(l[1])

bad_count=0
triples=[]
with open(os.path.join('../../KgCLUE-main/knowledge/', 'Knowledge.txt')) as f:
    lines=f.readlines()
for i in tqdm(range(len(lines))):
    l = lines[i].strip().split('|||')
    try:
        s = ent2id[l[0].strip()]
        p = rel2id[l[1].strip()]
        o = ent2id[l[2].strip()]
        triples.append((s, p, o)) 
    except:
        bad_count+=1


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22884223/22884223 [00:23<00:00, 957400.90it/s]


In [33]:
bad_count

688

In [35]:
relations_set=set()
entities_set=set()

for s,p,o in triples:
    entities_set.add(s)
    entities_set.add(o)
    relations_set.add(p)
    
print(len(entities_set))
print(len(relations_set))
bad_rel=0
with open("../../KgCLUE-main/knowledge/relations.dict",'w') as f:
    for i,ent in enumerate(relations_set):
        if ent=='' or ent=='\n':
            bad_rel+=1
            continue
        f.write(str(ent)+'\t'+str(i)+'\n')
        
bad_ent=0
with open("../../KgCLUE-main/knowledge/entities.dict",'w') as f:
    for i,ent in enumerate(entities_set):
        if ent=='' or ent=='\n':
            bad_ent+=1
            continue
        f.write(str(ent)+'\t'+str(i)+'\n')

9418099
245837


大规模图谱中一共有22884223个三元组，9418365个实体，245838个关系

In [38]:
bad_ent

0

In [18]:
'|||'.join('\t'.split(lines[i].strip()))

'\t'

In [40]:
ent2id['!=']

2225677

In [42]:
len(triples)

22883535

In [44]:
with open(os.path.join('../../KgCLUE-main/knowledge/', 'Knowledge.txt'),'w') as f:
    for s,p,o in triples:
        f.write('|||'.join([str(s),str(p),str(o)])+'\n')